In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

#import algorithms
from xgboost import XGBClassifier

# NLP Packages
import pickle
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

#Importing Libraries
#import pyforest
%time
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
import heapq # <-- Efficient sorting of large lists

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from random import randint
import sys
import pylab as pl
import spacy

# Entity featurization and similarity computation
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import utils
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics import classification_report, confusion_matrix 
import sklearn.model_selection as model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC 

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
/kaggle/input/edsa-movie-recommender-challenge-2022/sample_submission.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/imdb_data.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_scores.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/links.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/sample_submission.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/imdb_data.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_tags.csv
/kaggle/input/edsa-movie-recommender-challenge-2022/genome_scores.csv
/kaggle/

**1. Loading the data**

In [2]:
movies_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/movies.csv')
imdb_data_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/imdb_data.csv')
genome_tags_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/genome_tags.csv')
genome_scores_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/genome_scores.csv')
train_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/train.csv')
test_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/test.csv')
tags_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/tags.csv')
links_df = pd.read_csv('../input/edsa-movie-recommender-challenge-2022/links.csv')

**2. Basic data checks
   2.1. checking the null columns in all datasets
   2.2. checking the shape of all datasets**
   

In [3]:
#checking the null columns
print (f'The number of nulls in the movies dataset is : {movies_df.isnull().sum().sum()}')
print (f'The number of nulls in the train dataset is : {train_df.isnull().sum().sum()}')
print (f'The number of nulls in the test dataset is : {test_df.isnull().sum().sum()}')
print (f'The number of nulls in the imdb dataset is : {imdb_data_df.isnull().sum().sum()}')
print (f'The number of nulls in the genome tags dataset is : {genome_tags_df.isnull().sum().sum()}')
print (f'The number of nulls in the genome scores dataset is : {genome_scores_df.isnull().sum().sum()}')
print (f'The number of nulls in the tags dataset is : {tags_df.isnull().sum().sum()}')
print (f'The number of nulls in the links dataset is : {links_df.isnull().sum().sum()}')

print ()

#checking the shape of the data
print ('checking the shapes of all datasets')
print (f'The shape of movies dataset: {movies_df.shape}')
print (f'The shape of train dataset: {train_df.shape}')
print (f'The shape of test dataset: {test_df.shape}')
print (f'The shape of imdb dataset: {imdb_data_df.shape}')
print (f'The shape of genome dataset: {genome_tags_df.shape}')
print (f'The shape of genome_scores dataset: {genome_scores_df.shape}')
print (f'The shape of tags dataset: {tags_df.shape}')
print (f'The shape of links dataset: {links_df.shape}')

The number of nulls in the movies dataset is : 0
The number of nulls in the train dataset is : 0
The number of nulls in the test dataset is : 0
The number of nulls in the imdb dataset is : 62481
The number of nulls in the genome tags dataset is : 0
The number of nulls in the genome scores dataset is : 0
The number of nulls in the tags dataset is : 16
The number of nulls in the links dataset is : 107

checking the shapes of all datasets
The shape of movies dataset: (62423, 3)
The shape of train dataset: (10000038, 4)
The shape of test dataset: (14310, 2)
The shape of imdb dataset: (27278, 6)
The shape of genome dataset: (1128, 2)
The shape of genome_scores dataset: (15584448, 3)
The shape of tags dataset: (1093360, 4)
The shape of links dataset: (62423, 3)


In [4]:
#removing the timestamp col from the train_df
train_df.drop(['timestamp'], axis=1, inplace=True)

In [5]:
#declaring cols to use going forward
#train_df.columns = ['userId', 'movieId', 'rating']
df = train_df
df.columns = ['userID', 'item', 'rating']

In [6]:
#basic check-up of the train_df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000038 entries, 0 to 10000037
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   userID  int64  
 1   item    int64  
 2   rating  float64
dtypes: float64(1), int64(2)
memory usage: 228.9 MB


In [7]:
#basic check-up of the train_df
print('Dataset shape: {}'.format(df.shape))

#checking the train_df dataset at a 20,000 line interval
print('-Dataset examples-')
print(df.iloc[::20000, :])

Dataset shape: (10000038, 3)
-Dataset examples-
          userID   item  rating
0           5163  57669     4.0
20000      26027   2710     4.0
40000      24869   2890     5.0
60000     115864   4975     0.5
80000      35798   1617     4.0
...          ...    ...     ...
9920000    79689   3699     3.0
9940000   119895    435     2.5
9960000   145621   4973     2.5
9980000   125156   2628     3.0
10000000   89307  96588     1.0

[501 rows x 3 columns]


**3. EDA
     3.1. Ratings Distribution
     3.2. Ratings Distribution By movie
     3.3. Top 15 sum of Ratings
     3.4. Ratings Distribution By User
     3.5. Dimensionality Reduction**

In [8]:
#EDA - 3.1. Ratings Distribution Graph
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = df['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,)
# Create layout
layout = dict(title = 'Distribution Of {} ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [9]:
#EDA - 3.2. Number of ratings per movie (The first 150)
data = df.groupby('item')['rating'].count().clip(upper=150)

# Create trace
trace = go.Histogram(x = data.values,name = 'Ratings', xbins = dict(start = 0,end = 150,size = 2))

# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per movieId (Clipped at 150)',
                   xaxis = dict(title = 'Number of Ratings Per movieId'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [10]:
#EDA - 3.3. Top 15 ratings by movie
train_df.groupby('item')['rating'].count().reset_index().sort_values('rating', ascending=False)[:15]

,item,rating
314,318,32831
351,356,32383
292,296,31697
585,593,29444
2479,2571,29014
257,260,27560
475,480,25518
522,527,24004
108,110,23722
2866,2959,23536


In [11]:
#EDA - 3.4. Number of ratings per user (the first 150 ratings)
data = train_df.groupby('userID')['rating'].count().clip(upper=150)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 150,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 150)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [12]:
#EDA - 3.5. Dimensionality Reduction - we will use 850 a benchmark for both user and movie ratings
# in other words all userIds that have less than 850, and also all movieIds with les than 850 ratings will be removed
min_ratings = 850
filtered_movieIds = train_df['item'].value_counts() > min_ratings
filtered_movieIds = filtered_movieIds[filtered_movieIds].index.tolist()

min_user_ratings = 850
filtered_userIds = train_df['userID'].value_counts() > min_user_ratings
filtered_userIds = filtered_userIds[filtered_userIds].index.tolist()

#creating the new df, called new_df
df_new = train_df[(train_df['item'].isin(filtered_movieIds)) & (train_df['userID'].isin(filtered_userIds))]
print('The original data frame shape:\t{}'.format(train_df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(10000038, 3)
The new data frame shape:	(186999, 3)


df_new = train_df[:100000]
df_new.columns = ['userID', 'item', 'rating']
reader = Reader(rating_scale=(df_new['rating'].min(), df_new['rating'].max()))
data = Dataset.load_from_df(df_new[['userID', 'item', 'rating']], reader)

min_ratings = 950
filter_items = df['item'].value_counts() > min_ratings
filter_items = filter_items[filter_items].index.tolist()

min_user_ratings = 950
filter_users = df['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['item'].isin(filter_items)) & (df['userID'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

**4. Modelling and Prediction
   4.1. Scaling the Ratings (using suprise's Reader)
   4.2. Individually creating the algos
   4.3. printing each algo's RMSE and MAE - for checking
   4.4. Training and fitting each model
   4.5. Saving the model
   4.6. Load the saved model
   4.7. creating a user defined function for predicitng/recommendations
   **


In [13]:
#Modeling - 4.1. Modeling preprocessing process
# 4.1.1. scaling the rating using surprise's Reader
reader = Reader(rating_scale=(df_new['rating'].min(), df_new['rating'].max()))

#4.1.2. creating a new 'new' df (with reduced number of rows)
data = Dataset.load_from_df(df_new[['userID', 'item', 'rating']], reader)

In [14]:
#4.2. individually creating the algos
SVD_algo = SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)
KNNBaseline_algo = KNNBaseline(k=40, min_k=1, sim_options={}, bsl_options={}, verbose=True)

In [15]:
#4.3. printing each algo's RMSE and MAE - for checking
print('4.3.1. creating the  RMSE and MAE for SVD_algo...')
print(cross_validate(SVD_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True))
print('finished with the  RMSE and MAE creation for SVD_algo...')

#print()
#print('creating the  RMSE and MAE for SVDpp_algo...')
#print(cross_validate(SVDpp_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True))
#print('finished with the  RMSE and MAE creation for SVDpp_algo...')

print('4.3.2. creating the RMSE and MAE creation for KNNBaseline_algo...')
print(cross_validate(KNNBaseline_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True))
print('finished with the  RMSE and MAE creation for KNNBaseline_algo...')
print()

print('finished with the RMSE and MAE for each algo...')

4.3.1. creating the  RMSE and MAE for SVD_algo...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7993  0.8009  0.8033  0.8073  0.8095  0.8041  0.0038  
MAE (testset)     0.6085  0.6099  0.6104  0.6164  0.6176  0.6126  0.0037  
Fit time          9.70    9.96    9.80    10.19   9.76    9.88    0.18    
Test time         0.24    0.25    0.54    0.25    0.25    0.31    0.12    
{'test_rmse': array([0.79934412, 0.80086578, 0.80326723, 0.80729172, 0.80952024]), 'test_mae': array([0.60854321, 0.60994013, 0.61042011, 0.6163653 , 0.61758542]), 'fit_time': (9.698051929473877, 9.957760095596313, 9.804887533187866, 10.187989711761475, 9.755136489868164), 'test_time': (0.2440474033355713, 0.2475743293762207, 0.5441396236419678, 0.2455463409423828, 0.24833011627197266)}
finished with the  RMSE and MAE creation for SVD_algo...
4.3.2. creating the RMSE and MAE creation for KNNBaseline_algo...
Estimati

In [16]:
#4.4. Training and fitting each model
from datetime import datetime
reader = Reader(rating_scale=(0.5, 5))

print("\n\n\t\t COUNT START in seconds\n\n")
start = datetime.now()

print("> Loading data...")
data = Dataset.load_from_df(df_new[['userID', 'item', 'rating']], reader)
print("> Finished loading the data")

print("> Creating trainset...")
trainset = data.build_full_trainset()
print("> Finished creating the trainset")

startTraining = datetime.now()
print("> Starting the training process...")
print("> Starting to train the SVD_algo...")
SVD_pred = SVD_algo.fit(trainset)
print("> Finished with SVD_algo...")

print()
print("> Starting to train the KNNBaseline_algo...")
KNNBaseline_pred = KNNBaseline_algo.fit(trainset)
print("> Fininshed with KNNBaseline_algo...")
print()

endTraining = datetime.now()
print("> OK \t\t It took: ", (endTraining-startTraining).seconds, "seconds")

end = datetime.now()
print (">> DONE \t\t It took", (end-start).seconds, "seconds" )



		 COUNT START in seconds


> Loading data...
> Finished loading the data
> Creating trainset...
> Finished creating the trainset
> Starting the training process...
> Starting to train the SVD_algo...
> Finished with SVD_algo...

> Starting to train the KNNBaseline_algo...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
> Fininshed with KNNBaseline_algo...

> OK 		 It took:  15 seconds
>> DONE 		 It took 15 seconds


In [17]:
#4.5. Saving the model
from surprise import dump
import os
model_filename = "./model.pickle"
print (">> Starting dump")

# Dump algorithm and reload it.
file_name = os.path.expanduser(model_filename)
dump.dump(file_name, algo=SVD_algo)
print (">> Dump done")
print(model_filename)

>> Starting dump
>> Dump done
./model.pickle


In [18]:
#4.6. Load the saved model
def load_model(model_filename):
    print (">> Loading dump")
    from surprise import dump
    import os
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    print (">> Loaded dump")
    return loaded_model

In [19]:
#4.7. creating a user defined function for predicitng/recommendations
# predicitng
from pprint import pprint as pp
model_filename = "./model.pickle"
def itemRating(user, item):
    uid = str(user)
    iid = str(item) 
    loaded_model = load_model(model_filename)
    prediction = loaded_model.predict(user, item, verbose=True)
    rating = prediction.est
    details = prediction.details
    uid = prediction.uid
    iid = prediction.iid
    true = prediction.r_ui
    ret = {
        'user': user, 
        'item': item, 
        'rating': rating, 
        'details': details,
        'uid': uid,
        'iid': iid,
        'true': true
        }
    pp (ret)
    print ('\n\n')
    return ret

In [20]:
df_new.head()

,userID,item,rating
21,95075,41571,3.0
62,75830,2713,4.0
116,24692,588,4.5
140,148291,8810,0.5
252,19886,4388,3.0


In [21]:
predictions = []
for i, row in test_df.iterrows():
    x = (SVD_pred.predict(row.userId, row.movieId))
    pred = x[3]
    predictions.append(pred)
    
test_df['id'] = test_df['userId'].map(str) +'_'+ test_df['movieId'].map(str)

results = pd.DataFrame({"id":test_df['id'],"rating": predictions})
results.to_csv("submission.csv", index=False)